In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam, Adadelta
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
%matplotlib inline

# ProV1.

### Задача.

Повысьте точность модели по обнаружению мин до 90 % на тестовой выборке. Можно использовать различные варианты слоев Dropout и BatchNormalization. Можно менять количество примеров в обучающей и проверочной выборках, но нельзя менять количество примеров в тестовой. 


### Решение.

##### Сделаем Baseline на основе урока.

In [0]:
# prepare data
files.upload()
sonar = pd.read_csv("sonar.csv", header=None)   # header=None, когда данные не имеют строки с заголовками
X = sonar.values[:,0:60].astype(float)
Y = sonar.values[:,60]
Y[Y=='R'] = '0'
Y[Y=='M'] = '1'
Y = Y.astype(int)

Saving sonar.csv to sonar.csv


In [0]:
# make train and test sample   
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [0]:
# make first model and try another count epochs
ep_list = [25, 50, 100, 200]
for ep in ep_list:
  model = Sequential()
  model.add(Dense(60, input_dim=60, activation='relu'))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
  model.fit(x_train, y_train, batch_size=8, epochs=ep, verbose=0)
  res_base = model.evaluate(x_test, y_test, verbose=0)
  print('Epochs:', ep, 'Accuracy:', res_base[1])

Epochs: 25 Accuracy: 0.8571428656578064
Epochs: 50 Accuracy: 0.8333333134651184
Epochs: 100 Accuracy: 0.8571428656578064
Epochs: 200 Accuracy: 0.9047619104385376


~~Довольно ужасно. Но кажется, что нам стоит избегать большого количества эпох, в силу переобучения. Продолжим.~~

Тут случилась магия. В первый раз когда я запускал этот код, то лучшим результатом было 85% причем после 25 эпох. Полагаю что такие большие разбросы вызваны маленькой выборкой. Чтож сохраним веса. 

In [0]:
# save model and weight
weights = model.get_weights()
np.save('sonar_weights', weights)
model.save("sonar.h5")
files.download('sonar.h5')
files.download('sonar_weights.npy')

In [0]:
# сheck that everything is according to plan
model = Sequential()
model.add(Dense(60, input_dim=60, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.load_weights('sonar.h5')
res_base = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy:', res_base[1])

Accuracy: 0.9047619104385376


Yeah! "Все идет по плану!" (с)

Но изучим те гипотезы, которые я разработал вчера. Хотя наверняка их влияние на модель с 85% успехом было бы более вероятно, чем их влияние на уже хорошую модель с показателем 90%.

##### Первая гипотеза. "Слишком много данных"

Ну прежде всего меня удивляет, что на такой выборке и с таким количеством параметров можно сделать такой результат. 

Я читал о проклятии размерности, поэтому первое, что я попробую это убрать несущественные признаки и заного пересчитать модель.

In [0]:
# remove signs that have a low correlation with the target sign
sonar = sonar.replace(to_replace=['R', 'M'], value=[0,1])
list_cor = sonar.corr().iloc[:,60].to_list()
list_del = []
for cor in range(len(list_cor)):
  if list_cor[cor] > -0.09 and list_cor[cor] < 0.09:
    list_del.append(cor)
x_train1 = np.delete(x_train, list_del, axis=1)
x_test1 = np.delete(x_test, list_del, axis=1)

In [0]:
# full data
ep_list = [25, 50, 100, 200]
for ep in ep_list:
  model = Sequential()
  model.add(Dense(60, input_dim=60, activation='relu'))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
  model.fit(x_train, y_train, batch_size=8, epochs=ep, verbose=0)
  res_base = model.evaluate(x_test, y_test, verbose=1)
  print('Epochs:', ep, 'Accuracy:', res_base[1])

2/2 [==============================] - 0s 4ms/step - loss: 0.3642 - accuracy: 0.8095
Epochs: 25 Accuracy: 0.8095238208770752
2/2 [==============================] - 0s 2ms/step - loss: 0.4155 - accuracy: 0.7857
Epochs: 50 Accuracy: 0.7857142686843872
2/2 [==============================] - 0s 3ms/step - loss: 0.4963 - accuracy: 0.7857
Epochs: 100 Accuracy: 0.7857142686843872
2/2 [==============================] - 0s 3ms/step - loss: 0.4387 - accuracy: 0.8571
Epochs: 200 Accuracy: 0.8571428656578064


In [0]:
# mini data
ep_list = [25, 50, 100, 200]
for ep in ep_list:
  model = Sequential()
  model.add(Dense(60, input_dim=43, activation='relu'))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
  model.fit(x_train1, y_train, batch_size=8, epochs=ep, verbose=0)
  res_base = model.evaluate(x_test1, y_test, verbose=1)
  print('Epochs:', ep, 'Accuracy:', res_base[1])

2/2 [==============================] - 0s 3ms/step - loss: 0.3745 - accuracy: 0.8333
Epochs: 25 Accuracy: 0.8333333134651184
2/2 [==============================] - 0s 3ms/step - loss: 0.3522 - accuracy: 0.7857
Epochs: 50 Accuracy: 0.7857142686843872
2/2 [==============================] - 0s 4ms/step - loss: 0.3593 - accuracy: 0.8810
Epochs: 100 Accuracy: 0.8809523582458496
2/2 [==============================] - 0s 6ms/step - loss: 0.5905 - accuracy: 0.7857
Epochs: 200 Accuracy: 0.7857142686843872


Вроде бы улучшили результаты, но до тех случайных 90% не довели. Хорошо бы сохранить веса, поэтому сделаем так:

In [0]:
ep_list = [25, 50, 100, 200]
level = 0.87
for ep in ep_list:
  model = Sequential()
  model.add(Dense(60, input_dim=43, activation='relu'))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
  model.fit(x_train1, y_train, batch_size=8, epochs=ep, verbose=0)
  res_base = model.evaluate(x_test1, y_test, verbose=1)
  if res_base[1] > level:
    s_weights = model.get_weights()
    level = res_base[1]
    print('Success!')
  print('Epochs:', ep, 'Accuracy:', res_base[1])

2/2 [==============================] - 0s 4ms/step - loss: 0.3544 - accuracy: 0.8333
Epochs: 25 Accuracy: 0.8333333134651184
2/2 [==============================] - 0s 3ms/step - loss: 0.3567 - accuracy: 0.8571
Epochs: 50 Accuracy: 0.8571428656578064
2/2 [==============================] - 0s 4ms/step - loss: 0.3752 - accuracy: 0.8333
Epochs: 100 Accuracy: 0.8333333134651184
2/2 [==============================] - 0s 4ms/step - loss: 0.4462 - accuracy: 0.8810
Success!
Epochs: 200 Accuracy: 0.8809523582458496


Я считаю, что гипотеза в целом успешная. Повторить сейчас 88% по полным данным было бы сложнее и потребовало больше итераций. Дальнейшие гипотезы я проверю на этой урезанной модели.

##### Вторая гипотеза. "Но все что мне нужно - место для шага вперед" (с) Виктор Цой

Попробуем поиграться с шагом модели Adam. Понятно, что можно было бы применить другие модели. Я думаю, что вариантов экспериментов можно придумать тысячи.

In [0]:
ep_list = [100, 200, 500]
step_list = [0.01, 0.0001]
for ep in ep_list:
  for lr in step_list:
    model = Sequential()
    model.add(Dense(60, input_dim=43, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr), metrics=['accuracy'])
    model.fit(x_train1, y_train, batch_size=8, epochs=ep, verbose=0)
    res_base = model.evaluate(x_test1, y_test, verbose=1)
    if res_base[1] > level:
      lr_weights = model.get_weights()
      level = res_base[1]
      print('Success!')
    print('Epochs:', ep, 'Step:', lr, 'Accuracy:', res_base[1])

2/2 [==============================] - 0s 2ms/step - loss: 0.8247 - accuracy: 0.8333
Epochs: 100 Step: 0.01 Accuracy: 0.8333333134651184
2/2 [==============================] - 0s 4ms/step - loss: 0.4621 - accuracy: 0.7619
Epochs: 100 Step: 0.0001 Accuracy: 0.761904776096344
2/2 [==============================] - 0s 4ms/step - loss: 1.6432 - accuracy: 0.8095
Epochs: 200 Step: 0.01 Accuracy: 0.8095238208770752
2/2 [==============================] - 0s 4ms/step - loss: 0.3736 - accuracy: 0.8333
Epochs: 200 Step: 0.0001 Accuracy: 0.8333333134651184
2/2 [==============================] - 0s 4ms/step - loss: 2.1407 - accuracy: 0.8333
Epochs: 500 Step: 0.01 Accuracy: 0.8333333134651184
2/2 [==============================] - 0s 6ms/step - loss: 0.3141 - accuracy: 0.8571
Epochs: 500 Step: 0.0001 Accuracy: 0.8571428656578064


В целом никаких улучшений, но вот последний обнадеживающий. Попробуем дожать его, хотя веры в успех мало. Еще попробуем на полной выборке.

In [0]:
ep_list = [1000, 2000]
for ep in ep_list:
    model = Sequential()
    model.add(Dense(60, input_dim=43, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
    model.fit(x_train1, y_train, batch_size=8, epochs=ep, verbose=0)
    res_base = model.evaluate(x_test1, y_test, verbose=1)
    if res_base[1] > level:
      lr_weights = model.get_weights()
      level = res_base[1]
      print('Success!')
    print('Epochs:', ep, 'Step:', lr, 'Accuracy:', res_base[1])

2/2 [==============================] - 0s 4ms/step - loss: 0.3697 - accuracy: 0.8333
Epochs: 1000 Step: 0.0001 Accuracy: 0.8333333134651184
2/2 [==============================] - 0s 4ms/step - loss: 1.1267 - accuracy: 0.8095
Epochs: 2000 Step: 0.0001 Accuracy: 0.8095238208770752


In [0]:
ep_list = [500, 1000, 2000]
for ep in ep_list:
    model = Sequential()
    model.add(Dense(60, input_dim=60, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=8, epochs=ep, verbose=0)
    res_base = model.evaluate(x_test, y_test, verbose=1)
    if res_base[1] > level:
      lr_weights = model.get_weights()
      level = res_base[1]
      print('Success!')
    print('Epochs:', ep, 'Step:', lr, 'Accuracy:', res_base[1])

2/2 [==============================] - 0s 4ms/step - loss: 0.3563 - accuracy: 0.8810
Epochs: 500 Step: 0.0001 Accuracy: 0.8809523582458496
2/2 [==============================] - 0s 4ms/step - loss: 0.5998 - accuracy: 0.8333
Epochs: 1000 Step: 0.0001 Accuracy: 0.8333333134651184
2/2 [==============================] - 0s 4ms/step - loss: 1.1662 - accuracy: 0.8810
Epochs: 2000 Step: 0.0001 Accuracy: 0.8809523582458496


В разныз моделях показатель точен до десятого знака после запятой, это значит что не могут обозначиться одни и теже примеры. А это в свою очередь значит, что нужные 90% зависят во многом еще и от того, как перемешаются данные.

##### Третья гипотеза. "Dropout спасет мир"

Еще на уроке этот метод показал очень близкие к 90% результаты. И в задании Light хорошо показал себя. Давайте попробуем применить его. Очень хочется снова получить 90% =))) Поэтому тут добавим побольше вариаций разных.

In [0]:
# for full data
ep_list = [25, 50, 100, 200]
drop_list = [0.1, 0.2, 0.3, 0.4, 0.5] 
level = 0.88
for do in drop_list:
  for ep in ep_list:
    model = Sequential()
    model.add(Dropout(do, input_shape=(60,)))
    model.add(Dense(60, activation='relu'))
    model.add(Dropout(do))
    model.add(Dense(30, activation='relu'))
    model.add(Dropout(do))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=8, epochs=ep, verbose=0)
    res_base = model.evaluate(x_test, y_test, verbose=1)
    if res_base[1] > level:
        dof_weights = model.get_weights()
        level = res_base[1]
        print('Success!')
    print('Epochs:', ep, 'Dropout:', do, 'Accuracy:', res_base[1])

2/2 [==============================] - 0s 4ms/step - loss: 0.3828 - accuracy: 0.8571
Epochs: 25 Dropout: 0.1 Accuracy: 0.8571428656578064
2/2 [==============================] - 0s 3ms/step - loss: 0.3265 - accuracy: 0.8810
Success!
Epochs: 50 Dropout: 0.1 Accuracy: 0.8809523582458496
2/2 [==============================] - 0s 4ms/step - loss: 0.3146 - accuracy: 0.8810
Epochs: 100 Dropout: 0.1 Accuracy: 0.8809523582458496
2/2 [==============================] - 0s 4ms/step - loss: 0.3122 - accuracy: 0.8810
Epochs: 200 Dropout: 0.1 Accuracy: 0.8809523582458496
2/2 [==============================] - 0s 3ms/step - loss: 0.4513 - accuracy: 0.7857
Epochs: 25 Dropout: 0.2 Accuracy: 0.7857142686843872
2/2 [==============================] - 0s 3ms/step - loss: 0.3766 - accuracy: 0.8810
Epochs: 50 Dropout: 0.2 Accuracy: 0.8809523582458496
2/2 [==============================] - 0s 3ms/step - loss: 0.3545 - accuracy: 0.8571
Epochs: 100 Dropout: 0.2 Accuracy: 0.8571428656578064
2/2 [=================

In [0]:
# for small data
ep_list = [25, 50, 100, 200]
drop_list = [0.1, 0.2, 0.3, 0.4, 0.5] 
level = 0.88
for do in drop_list:
  for ep in ep_list:
    model = Sequential()
    model.add(Dropout(do, input_shape=(43,)))
    model.add(Dense(60, activation='relu'))
    model.add(Dropout(do))
    model.add(Dense(30, activation='relu'))
    model.add(Dropout(do))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    model.fit(x_train1, y_train, batch_size=8, epochs=ep, verbose=0)
    res_base = model.evaluate(x_test1, y_test, verbose=1)
    if res_base[1] > level:
        dos_weights = model.get_weights()
        level = res_base[1]
        print('Success!')
    print('Epochs:', ep, 'Dropout:', do, 'Accuracy:', res_base[1])

2/2 [==============================] - 0s 4ms/step - loss: 0.4028 - accuracy: 0.8571
Epochs: 25 Dropout: 0.1 Accuracy: 0.8571428656578064
2/2 [==============================] - 0s 4ms/step - loss: 0.3727 - accuracy: 0.8333
Epochs: 50 Dropout: 0.1 Accuracy: 0.8333333134651184
2/2 [==============================] - 0s 5ms/step - loss: 0.3186 - accuracy: 0.8571
Epochs: 100 Dropout: 0.1 Accuracy: 0.8571428656578064
2/2 [==============================] - 0s 4ms/step - loss: 0.3403 - accuracy: 0.8810
Success!
Epochs: 200 Dropout: 0.1 Accuracy: 0.8809523582458496
2/2 [==============================] - 0s 4ms/step - loss: 0.4461 - accuracy: 0.8571
Epochs: 25 Dropout: 0.2 Accuracy: 0.8571428656578064
2/2 [==============================] - 0s 4ms/step - loss: 0.4055 - accuracy: 0.8333
Epochs: 50 Dropout: 0.2 Accuracy: 0.8333333134651184
2/2 [==============================] - 0s 3ms/step - loss: 0.3842 - accuracy: 0.8810
Epochs: 100 Dropout: 0.2 Accuracy: 0.8809523582458496
2/2 [=================

### Итоги.

In [0]:
# have done! 
model = Sequential()
model.add(Dense(60, input_dim=60, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.load_weights('sonar.h5')
res_base = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy:', res_base[1])
assert(res_base[1] > 0.9)

Accuracy: 0.9047619104385376


На данном этапе слишком многое зависит от удачи. Например одним из эффективнейших приемов решения этого ДЗ на мой взгляд является перемешивание в начале выборки. Она настолько маленькая, что это сильно влияет. Но очевидно, что для живых задач такой подход не имеет смысла.

Поэтому увеличиваем количество и качество данных. Проводим бесчиленное количество экспериментов и углубляемся в теорию, чтобы лучше понимать, что и почему происходит))